## Importação das Bibliotecas

Nesta célula importamos todas as bibliotecas necessárias para o projeto.

- **PyTorch (`torch`, `torch.nn`, `torch.optim`)**: utilizado para criar, treinar e avaliar a rede neural do tipo Variational Autoencoder (VAE).
- **Pandas (`pd`)** e **NumPy (`np`)**: usados para manipulação e processamento dos dados.
- **Scikit-learn**:
  - `train_test_split`: para separar os dados em conjuntos de treino e teste.
  - `StandardScaler`: para normalizar as features numéricas.
- **Torch Utils (`DataLoader`, `TensorDataset`)**: facilitam o gerenciamento de dados em formato compatível com PyTorch.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

## Carregamento do Dataset

Aqui carregamos o dataset **Delaney (ESOL)**, que contém descritores moleculares e informações de solubilidade.

Esse dataset é amplamente utilizado em Química Computacional e Drug Discovery para tarefas de:
- Predição de propriedades moleculares
- Aprendizado de representações químicas

## Extração das Features

Aqui extraímos do DataFrame apenas as colunas selecionadas anteriormente e as convertemos para um array NumPy.

Esse formato é necessário para:
- Normalização dos dados
- Conversão posterior para tensores PyTorch

In [2]:
url = "https://raw.githubusercontent.com/UnB-CIS/UnB-CIS-UEG-MPhysChem/refs/heads/main/4_Dia/delaney-processed.csv"
df = pd.read_csv(url)

features = [
    "Minimum Degree",
    "Molecular Weight",
    "Number of H-Bond Donors",
    "Number of Rings",
    "Number of Rotatable Bonds",
    "Polar Surface Area"
]

X = df[features].values

## Normalização das Features

Utilizamos o `StandardScaler` para padronizar as features:
- Média igual a 0
- Desvio padrão igual a 1

A normalização é essencial para redes neurais, pois:
- Evita que features em escalas maiores dominem o treinamento
- Facilita a convergência do otimizador

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Split de Treino e Teste

Nesta célula dividimos os dados em:
- **80% para treinamento**
- **20% para teste**

O conjunto de teste será utilizado apenas na etapa final para avaliar
a capacidade de generalização do modelo.

In [4]:
X_train, X_test = train_test_split(
    X, test_size=0.2, random_state=42
)

## Conversão para Tensores

Aqui convertemos os arrays NumPy para tensores do PyTorch.

Isso é necessário porque:
- O PyTorch opera exclusivamente com tensores
- Permite cálculo automático de gradientes (autograd)

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)

## Definição do Modelo VAE

Nesta célula definimos a arquitetura do **Variational Autoencoder**.

O VAE é composto por:
- **Encoder**: comprime os dados de entrada em um espaço latente
- **Espaço Latente**: representado por média (`μ`) e variância (`σ²`)
- **Decoder**: reconstrói os dados originais a partir do espaço latente

Esse tipo de modelo é amplamente utilizado em:
- Aprendizado de representações
- Geração de moléculas
- Drug Discovery

In [6]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )

        self.mu_layer     = nn.Linear(16, latent_dim)
        self.logvar_layer = nn.Linear(16, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.mu_layer(h)
        logvar = self.logvar_layer(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

## Função de Perda do VAE

A função de perda do VAE possui dois componentes:

1. **Erro de Reconstrução (MSE)**  
   Mede o quão bem o modelo reconstrói os dados originais.

2. **Divergência KL**  
   Regulariza o espaço latente para que siga uma distribuição normal padrão.

Essa combinação garante:
- Boa reconstrução
- Espaço latente contínuo e bem estruturado

In [7]:
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.MSELoss()(recon_x, x)

    kl_div = -0.5 * torch.mean(
        1 + logvar - mu.pow(2) - logvar.exp()
    )

    return recon_loss + kl_div

## Inicialização do Modelo

Aqui definimos:
- Dimensão de entrada: número de features químicas
- Dimensão do espaço latente: 2 (para facilitar visualização)

Também inicializamos:
- O modelo VAE
- O otimizador Adam

In [8]:
input_dim = X_train.shape[1]
latent_dim = 2  # espaço latente 2D para visualização

model = VAE(input_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 200

## Treinamento do VAE

Nesta célula realizamos o treinamento do modelo.

Para cada época:
1. O modelo entra em modo de treinamento
2. Calcula a reconstrução e a loss
3. Executa backpropagation
4. Atualiza os pesos

A loss impressa reflete:
- Qualidade da reconstrução
- Organização do espaço latente

In [9]:
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    x_recon, mu, logvar = model(X_train)
    loss = vae_loss(x_recon, X_train, mu, logvar)

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")

Epoch 000 | Loss: 1.0335
Epoch 020 | Loss: 0.9883
Epoch 040 | Loss: 0.9797
Epoch 060 | Loss: 0.9765
Epoch 080 | Loss: 0.9767
Epoch 100 | Loss: 0.9743
Epoch 120 | Loss: 0.9732
Epoch 140 | Loss: 0.9717
Epoch 160 | Loss: 0.9737
Epoch 180 | Loss: 0.9464


## Avaliação no Conjunto de Teste

Aqui avaliamos o desempenho do modelo em dados nunca vistos.

- `model.eval()` desativa comportamentos específicos de treino
- `torch.no_grad()` evita o cálculo de gradientes

Isso nos dá uma estimativa realista da generalização do modelo.

In [10]:
model.eval()

with torch.no_grad():
    x_recon, mu_test, logvar_test = model(X_test)
    test_loss = vae_loss(x_recon, X_test, mu_test, logvar_test)

print("Test Loss:", round(test_loss.item(), 4))


Test Loss: 1.0664


## Extração dos Embeddings Latentes

Nesta etapa extraímos apenas o vetor de média (`μ`) do espaço latente
para todas as moléculas do dataset.

Esses embeddings podem ser usados para:
- Visualização
- Clusterização de moléculas
- Similaridade molecular
- Geração de novos compostos (Drug Discovery)

In [11]:
model.eval()

with torch.no_grad():
    _, mu_all, _ = model(torch.tensor(X, dtype=torch.float32))

latent_embeddings = mu_all.numpy()

In [12]:
latent_embeddings

array([[ 0.95040095,  3.063143  ],
       [ 0.04440671,  0.0448938 ],
       [-0.1410132 , -0.37670258],
       ...,
       [-0.08523327, -0.21272291],
       [-0.2755366 , -0.574624  ],
       [ 0.08835062,  0.17385277]], dtype=float32)

---

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

## Representação Molecular com SMILES

Nesta etapa, utilizamos a representação **SMILES (Simplified Molecular Input Line Entry System)** para descrever moléculas como sequências de caracteres.

Diferentemente dos descritores tabulares, SMILES preserva a **estrutura química explícita**, permitindo que o modelo aprenda padrões diretamente da linguagem molecular.


## Tokenização dos SMILES

Como redes neurais não operam diretamente sobre texto, os SMILES são **tokenizados**, ou seja, cada caractere químico é convertido em um símbolo discreto.

Cada token é mapeado para um índice numérico, criando um vocabulário químico que será usado pela rede neural.

## Codificação e Padding das Sequências

Os SMILES possuem comprimentos variáveis.  
Para permitir o processamento em batch, todas as sequências são:

- convertidas para índices numéricos
- preenchidas (*padding*) até um comprimento máximo comum

O token `<PAD>` é utilizado apenas para alinhamento e não carrega informação química.

In [14]:
def tokenize_smiles(smiles):
    return list(smiles)

smiles_list = df["smiles"].tolist()

tokens = sorted(set("".join(smiles_list)))
token2idx = {t: i+1 for i, t in enumerate(tokens)}
token2idx["<PAD>"] = 0
idx2token = {i: t for t, i in token2idx.items()}

In [17]:
max_len = max(len(s) for s in smiles_list)

def encode_smiles(smiles):
    seq = [token2idx[t] for t in tokenize_smiles(smiles)]
    return seq + [0] * (max_len - len(seq))

X_smiles = torch.tensor([encode_smiles(s) for s in smiles_list])

In [20]:
X_smiles.shape

torch.Size([1128, 98])

In [21]:
X_smiles

tensor([[21, 16, 16,  ...,  0,  0,  0],
        [16, 27,  6,  ...,  0,  0,  0],
        [16, 16,  3,  ...,  0,  0,  0],
        ...,
        [16, 16, 23,  ...,  0,  0,  0],
        [16, 16, 16,  ...,  0,  0,  0],
        [16, 21, 22,  ...,  0,  0,  0]])

## Arquitetura do Variational Autoencoder (VAE)

Aqui definimos a arquitetura do **SMILES-VAE**, composta por:

- **Encoder sequencial:** aprende uma representação compacta da molécula
- **Camadas de média (μ) e variância (log σ²):** definem a distribuição latente
- **Decoder sequencial:** reconstrói o SMILES a partir do espaço latente

Essa arquitetura permite aprender um **espaço latente contínuo e regularizado** para moléculas químicas.

## Reparametrização do Espaço Latente

O VAE utiliza o truque de **reparametrização** para permitir o treinamento via backpropagation.

Em vez de amostrar diretamente da distribuição, o modelo aprende:
- a média (μ)
- a variância (σ²)

e gera o vetor latente de forma diferenciável.

In [22]:
class SmilesVAE(nn.Module):
    def __init__(self, vocab_size, embed_dim, latent_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        self.encoder_rnn = nn.GRU(embed_dim, 128, batch_first=True)
        self.mu = nn.Linear(128, latent_dim)
        self.logvar = nn.Linear(128, latent_dim)

        self.decoder_rnn = nn.GRU(embed_dim, 128, batch_first=True)
        self.decoder_out = nn.Linear(128, vocab_size)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        emb = self.embedding(x)
        _, h = self.encoder_rnn(emb)
        h = h.squeeze(0)

        mu = self.mu(h)
        logvar = self.logvar(h)

        z = self.reparameterize(mu, logvar)
        z = z.unsqueeze(1).repeat(1, x.size(1), 1)

        dec_out, _ = self.decoder_rnn(z)
        logits = self.decoder_out(dec_out)

        return logits, mu, logvar

## Função de Perda do VAE

A função de perda do Variational Autoencoder possui dois componentes:

1. **Perda de reconstrução:**  
   Mede o quão bem o SMILES original é reconstruído pelo decoder.

2. **Divergência KL:**  
   Regulariza o espaço latente para se aproximar de uma distribuição normal.

Essa combinação garante um espaço latente:
- contínuo
- suave
- adequado para geração de novas moléculas


In [23]:
def vae_loss(logits, x, mu, logvar):
    recon = nn.CrossEntropyLoss(ignore_index=0)(
        logits.view(-1, logits.size(-1)),
        x.view(-1)
    )

    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + kl

## Treinamento do SMILES-VAE

Durante o treinamento, o modelo aprende simultaneamente:
- a reconstruir SMILES válidos
- a organizar o espaço latente químico

O processo ajusta os parâmetros do encoder e decoder para minimizar a perda total do VAE.

## Extração de Embeddings Moleculares

Após o treinamento, utilizamos apenas o **encoder** para extrair os vetores latentes (μ).

Esses vetores representam **embeddings moleculares contínuos**, que podem ser usados para:
- clustering químico
- busca de moléculas similares
- regressão de propriedades
- geração de novos candidatos a fármacos


In [24]:
model = SmilesVAE(
    vocab_size=len(token2idx),
    embed_dim=32,
    latent_dim=32
)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    optimizer.zero_grad()
    logits, mu, logvar = model(X_smiles)
    loss = vae_loss(logits, X_smiles, mu, logvar)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss {loss.item():.4f}")

Epoch 0 | Loss 3.4992
Epoch 10 | Loss 3.1679
Epoch 20 | Loss 2.5440
Epoch 30 | Loss 2.4811
Epoch 40 | Loss 2.4259
Epoch 50 | Loss 2.4102
Epoch 60 | Loss 2.4018
Epoch 70 | Loss 2.3820
Epoch 80 | Loss 2.3773
Epoch 90 | Loss 2.3638


## Conexão com Drug Discovery

O espaço latente aprendido pelo SMILES-VAE permite aplicações diretas em Drug Discovery, como:

- geração de novas moléculas
- otimização de propriedades (ADMET, solubilidade, afinidade)
- exploração química guiada por aprendizado profundo

Essa abordagem está na base de pipelines modernos de descoberta de fármacos baseados em IA.
